## <a href = 'https://8weeksqlchallenge.com/case-study-2/'>Pizza Runner</a>

#### Introduction

<img src = 'https://8weeksqlchallenge.com/images/case-study-designs/2.png'>

<br>

Did you know that over 115 million kilograms of pizza is consumed daily worldwide??? (Well according to Wikipedia anyway…)

Danny was scrolling through his Instagram feed when something really caught his eye - “80s Retro Styling and Pizza Is The Future!”

Danny was sold on the idea, but he knew that pizza alone was not going to help him get seed funding to expand his new Pizza Empire - so he had one more genius idea to combine with it - he was going to Uberize it - and so Pizza Runner was launched!

Danny started by recruiting “runners” to deliver fresh pizza from Pizza Runner Headquarters (otherwise known as Danny’s house) and also maxed out his credit card to pay freelance developers to build a mobile app to accept orders from customers.

#### Case Study Questions - Part B

1. How many runners signed up for each 1 week period? (i.e. week starts 2021-01-01)
2. What was the average time in minutes it took for each runner to arrive at the Pizza Runner HQ to pickup the order?
3. Is there any relationship between the number of pizzas and how long the order takes to prepare?
4. What was the average distance travelled for each customer?
5. What was the difference between the longest and shortest delivery times for all orders?
6. What was the average speed for each runner for each delivery and do you notice any trend for these values?
7. What is the successful delivery percentage for each runner?

In [1]:
import sql
from sqlalchemy import create_engine


%load_ext sql
%sql sqlite://

Tables as per link above

In [2]:
%%sql
DROP TABLE IF EXISTS runner_orders;
CREATE TABLE runner_orders("order_id" INTEGER,
                           "runner_id" INTEGER,
                           "pickup_time" VARCHAR(19),
                           "distance" VARCHAR(7),
                           "duration" VARCHAR(10),
                           "cancellation" VARCHAR(23)
);

INSERT INTO runner_orders
    ("order_id", "runner_id", "pickup_time", "distance", "duration", "cancellation")
VALUES
    (1, 1, "2021-01-01 18:15:34", "20km", "32 minutes", ""),
    (2, 1, "2021-01-01 19:10:54", "20km", "27 minutes", ""),
    (3, 1, "2021-01-03 00:12:37", "13.4km", "20 mins", NULL),
    (4, 2, "2021-01-04 13:53:03", "23.4", "40", NULL),
    (5, 3, "2021-01-08 21:10:57", "10", "15", NULL),
    (6, 3, null, null, null, "Restaurant Cancellation"),
    (7, 2, "2020-01-08 21:30:45", "25km", "25mins", null),
    (8, 2, "2020-01-10 00:15:02", "23.4 km", "15 minute", null),
    (9, 2, null, null, null, "Customer Cancellation"),
    (10, 1, "2020-01-11 18:50:20", "10km", "10minutes", null);

 * sqlite://
Done.
Done.
10 rows affected.


[]

In [21]:
%%sql
DROP TABLE IF EXISTS runners;
CREATE TABLE runners("runner_id" INTEGER,
                     "registration_date" DATE);
INSERT INTO runners
    ("runner_id", "registration_date")
VALUES
    (1, '2021-01-01'),
    (2, '2021-01-03'),
    (3, '2021-01-08'),
    (4, '2021-01-15');

 * sqlite://
Done.
Done.
4 rows affected.


[]

In [4]:
%%sql
DROP TABLE IF EXISTS customer_orders;
CREATE TABLE customer_orders("order_id" INTEGER,
                             "customer_id" INTEGER,
                             "pizza_id" INTEGER,
                             "exclusions" VARCHAR(4),
                             "extras" VARCHAR(4),
                             "order_time" TIMESTAMP);
INSERT INTO customer_orders
    ("order_id", "customer_id", "pizza_id", "exclusions", "extras", "order_time")
VALUES
    (1,  101, 1, " ", " ", "2021-01-01 18:05:02"),
    (2,  101, 1, " ", " ", "2021-01-01 19:00:52"),
    (3,  102, 1, " ", " ", "2021-01-02 23:51:23"),
    (3,  102, 2, " ", null, "2021-01-02 23:51:23"),
    (4,  103, 1, "4", " ", "2021-01-04 13:23:46"),
    (4,  103, 1, "4", " ", "2021-01-04 13:23:46"),
    (4,  103, 2, "4", " ", "2021-01-04 13:23:46"),
    (5,  104, 1, null, 1, "2021-01-08 21:00:29"),
    (6,  101, 2, null, null, "2021-01-08 21:03:13"),
    (7,  105, 2, null, 1, "2021-01-08 21:20:29"),
    (8,  102, 1, null, null, "2021-01-09 23:54:33"),
    (9,  103, 1, "4", "1, 5", "2021-01-10 11:22:59"),
    (10, 104, 1, null, null, "2021-01-11 18:34:49"),
    (10, 104, 1, "2, 6", "1, 4", "2021-01-11 18:34:49");

 * sqlite://
Done.
Done.
14 rows affected.


[]

In [5]:
%%sql
DROP TABLE IF EXISTS pizza_names;
CREATE TABLE pizza_names ("pizza_id" INTEGER,
                          "pizza_name" TEXT);
INSERT INTO pizza_names ("pizza_id", "pizza_name")
VALUES 
    (1, 'Meatlovers'),
    (2, 'Vegetarian');

 * sqlite://
Done.
Done.
2 rows affected.


[]

In [6]:
%%sql
DROP TABLE IF EXISTS pizza_recipes;
CREATE TABLE pizza_recipes("pizza_id" INTEGER,
                           "toppings" TEXT);
INSERT INTO pizza_recipes("pizza_id", "toppings")
VALUES
    (1, "1, 2, 3, 4, 5, 6, 8, 10"),
    (2, "4, 6, 7, 9, 11, 12");

 * sqlite://
Done.
Done.
2 rows affected.


[]

In [7]:
%%sql
DROP TABLE IF EXISTS pizza_toppings;
CREATE TABLE pizza_toppings("topping_id" INTEGER,
                            "topping_name" TEXT);
INSERT INTO pizza_toppings("topping_id", "topping_name")
VALUES
    (1, "Bacon"),
    (2, "BBQ Sauce"),
    (3, "Beef"),
    (4, "Cheese"),
    (5, "Chicken"),
    (6, "Mushrooms"),
    (7, "Onions"),
    (8, "Pepperoni"),
    (9, "Peppers"),
    (10, "Salami"),
    (11, "Tomatoes"),
    (12, "Tomato Sauce");

 * sqlite://
Done.
Done.
12 rows affected.


[]

1. How many runners signed up for each 1 week period? (i.e. week starts 2021-01-01)

In [34]:
%%sql

SELECT 
    strftime('%W', registration_date) AS Week_number,
    COUNT(runner_id) AS signups_this_week
FROM runners 
GROUP BY Week_number;

 * sqlite://
Done.


Week_number,signups_this_week
00,2
01,1
02,1


2. What was the average time in minutes it took for each runner to arrive at the Pizza Runner HQ to pickup the order?

pickup time - order time.

In [ ]:
%%sql

SELECT 